In [13]:
import numpy as np
import os

In [1]:
import sys
import autoreload
from Frames_Alignment import *
%load_ext autoreload
%autoreload 2

In [3]:
def Dlib_Stability_Check(file,batch_size,cmap,resize,min_size):
    landmarks_height = []
    
    # 帧生成器
    frames_gen = yield_frames_through_idx(file,batch_size,min_size,resize)
    
    while True:
        frames = next(frames_gen)
        if len(frames) == 0:
            break
        
        #  获取帧中的人脸位置及关键点坐标
        batch_of_face_locations,batch_of_landmarks,valid = get_location_landmark(frames,0)
        
        if not valid:
            continue
        
        #  获取脸颊关键点，眼睛关键点的个数
        num_chin = len(batch_of_landmarks[0][0]['chin'])
        num_eye = len(batch_of_landmarks[0][0]['left_eye'])
        
        for landmarks in batch_of_landmarks:
            if len(landmarks) != 1:
                continue
            left_eye_center_y = np.mean([y for x,y in landmarks[0]['left_eyebrow']])
            right_eye_center_y = np.mean([y for x,y in landmarks[0]['right_eyebrow']])
            eye_center_y = np.mean([left_eye_center_y,right_eye_center_y])
            bottom_chin_y = np.max([y for x,y in landmarks[0]['chin']])
            
            landmarks_height.append(bottom_chin_y - eye_center_y)
        
        #  如果frames长度不足batch_size，说明视频读完了
        if len(frames) != batch_size:
            break
        
    return landmarks_height

## 下面有两个数据集VIPL-HR以及BEAN-HR的处理过程  
## 只需要选择运行一种数据集下的所有代码就行

## VIPL-HR数据集

In [4]:
batch_size = 10
cmap = 'RGB'
resize = False
remove_skin = True
min_size = None
save_root_path = './Plot-Analysis/Dlib-Stability-Check'
dataset_source = "VIPL"

### 注意下面的 **root_dir** 修改成自己电脑上存放VIPL-HR数据集的根目录路径  
### 数据集的申请参考[中科院官网](https://vipl.ict.ac.cn/view_database.php?id=15)
### 下载后的文件结构都是一致的

In [9]:
root_dir = "D:\BaiduNetdiskDownload\VIPL-HR\data"

"""测试者分组的路径"""
people_grouped_path = [os.path.join(root_dir,name) for name in os.listdir(root_dir) if 'zip' not in name]
num_grouped = len(people_grouped_path)

"""单个测试者的路径"""
person_path = []
for group in people_grouped_path:
    person_path.extend([os.path.join(group,name) for name in os.listdir(group)])
num_person = len(person_path)

"""所有场景的路径；不寻常数量，即数目不为9个场景的测试者路径；以及所有可能的场景数目"""
scenario_path = []
abnormal_number_of_scenarios = []
possible_number_of_scenarios = set()
for person in person_path:
    scenario_path.extend([os.path.join(person,name) for name in os.listdir(person)])
    
    possible_number_of_scenarios.add(len(os.listdir(person)))
    if len(os.listdir(person)) != 9:
        abnormal_number_of_scenarios.append(person)

"""检查是否所有场景都有source2，结论：都有"""
for scenario in scenario_path:
    if 'source2' not in os.listdir(scenario):
        print(scenario)

def get_video_path_with_scene_source(scenario_path,scenario='v1',source='source2'):
    """
      获取特定场景和视频源的视频路径
    """
    target_scenario_path = [path for path in scenario_path if os.path.basename(path) == scenario]
    
    video_path = []
    for path in target_scenario_path:
        video_path.append(os.path.join(os.path.join(path,source),"video.avi"))
    
    return video_path

video_path_v1 = get_video_path_with_scene_source(scenario_path,scenario='v1',source='source2')  #  stable
video_path_v3 = get_video_path_with_scene_source(scenario_path,scenario='v3',source='source2')  #  stable and talking
video_path_v4 = get_video_path_with_scene_source(scenario_path,scenario='v4',source='source2')  #  stable and bit dark
video_path_v5 = get_video_path_with_scene_source(scenario_path,scenario='v5',source='source2')  #  stable and bit light
video_path_v6 = get_video_path_with_scene_source(scenario_path,scenario='v6',source='source2')  #  stable and long distance
video_path_v7 = get_video_path_with_scene_source(scenario_path,scenario='v7',source='source2')  #  stable after exercise
video_path_v8 = get_video_path_with_scene_source(scenario_path,scenario='v8',source='source2')  #  stable after exercise

all_video_path_VIPL = video_path_v1 + video_path_v3 + video_path_v4 + video_path_v5 + video_path_v6 + video_path_v7 + video_path_v8
# person_id_without_glasses = [12,14,18,19,23,26,28,34,38,39,40,41,48,52,57,58,63,65,67,70,71,74,79,84,85,91,92,95,97,99]

# all_video_path_without_glasses = [video for video in all_video_path_VIPL if int(video.split("\\")[-4][1:]) in person_id_without_glasses]

## BEAN-HR数据集

### 注意下面的 **video_root_path** 是我自己电脑上存放BEAN-HR数据集的根目录路径  
### 数据集的暂时不对外开放  

In [14]:
video_root_path = 'D:\\BaiduNetdiskDownload\\ECNU-Dataset-AVC1'
batch_size = 10
cmap = 'RGB'
resize = False
remove_skin = True
min_size = 1000
save_root_path = './Plot-Analysis/Dlib-Stability-Check'
dataset_source = "BEAN"

all_video_path_BEAN = []
for dirpath,_,filenames in os.walk(video_root_path):
    for file in filenames:
        all_video_path_BEAN.append(os.path.join(dirpath,file))

## 利用切割框的高的标准差衡量稳定性

In [ ]:
stds = []
for idx,video in enumerate(all_video_path_BEAN):
    print("The {}th video:".format(idx + 1))
    landmarks_height = Dlib_Stability_Check(video,batch_size,cmap,resize,min_size)
    print("The standard deviation of height is:",np.std(landmarks_height))
    stds.append(np.std(landmarks_height))

In [ ]:
if not os.path.exists(save_root_path):
    os.makedirs(save_root_path)
with open(os.path.join(save_root_path,"{}-HR.txt".format(dataset_source)),'w') as f:
    f.write(" ".join([str(std) for std in stds]))

plt.hist(stds, bins=61, facecolor="blue", edgecolor="black", alpha=0.7)
plt.xlabel("Standard deviation",fontsize=20)
plt.ylabel("Frequency",fontsize=20)
plt.xticks(np.arange(0,61,5),np.arange(0,61,5))
plt.tick_params(labelsize=15)

plt.tight_layout()
plt.savefig(os.path.join(save_root_path,"{}-HR.png".format(dataset_source)),dpi=300)
plt.show()